In [51]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization



In [137]:
data = '/Users/jj/Documents/COLLEGE_DOCS/CASME2/Coding with Computed LBP Features.csv'
df = pd.read_csv(data)

In [138]:
df.shape

(9, 8)

In [139]:
df.head()

,Subject,Filename,OnsetFrame,ApexFrame,OffsetFrame,Action Units,Estimated Emotion,Computed LBP
0,1,EP02_01f,46,59,86,12,happiness,1.0 3.0 4.0 5.0 5.0 3.0 3.0 0.0 5.0 4.0 1.0 1....
1,1,EP19_05f,396,416,446,4+L10,disgust,1.0 3.0 4.0 5.0 5.0 3.0 3.0 0.0 5.0 4.0 1.0 1....
2,1,EP19_06f,36,71,161,4+5+L10,disgust,3.0 4.0 4.0 4.0 4.0 5.0 4.0 3.0 3.0 4.0 3.0 4....
3,2,EP01_11f,46,91,96,15,repression,3.0 3.0 5.0 4.0 3.0 3.0 3.0 3.0 3.0 5.0 3.0 3....
4,2,EP02_04f,31,79,141,12+15,repression,3.0 3.0 3.0 3.0 3.0 3.0 3.0 3.0 3.0 3.0 3.0 3....


In [55]:
col_names = df.columns

col_names

Index(['Subject', 'Filename', 'OnsetFrame', 'ApexFrame', 'OffsetFrame',
       'Action Units', 'Estimated Emotion'],
      dtype='object')

In [140]:
X = df.drop(['Subject', 'Filename', 'Action Units', 'Estimated Emotion'], axis=1)
y = df['Estimated Emotion']

print(X)
print(y)

   OnsetFrame  ApexFrame  OffsetFrame   
0          46         59           86  \
1         396        416          446   
2          36         71          161   
3          46         91           96   
4          31         79          141   
5          16         81          136   
6          51         69           91   
7          21         81          106   
8          61         76          121   

                                        Computed LBP  
0  1.0 3.0 4.0 5.0 5.0 3.0 3.0 0.0 5.0 4.0 1.0 1....  
1  1.0 3.0 4.0 5.0 5.0 3.0 3.0 0.0 5.0 4.0 1.0 1....  
2  3.0 4.0 4.0 4.0 4.0 5.0 4.0 3.0 3.0 4.0 3.0 4....  
3  3.0 3.0 5.0 4.0 3.0 3.0 3.0 3.0 3.0 5.0 3.0 3....  
4  3.0 3.0 3.0 3.0 3.0 3.0 3.0 3.0 3.0 3.0 3.0 3....  
5  3.0 3.0 3.0 3.0 3.0 3.0 3.0 1.0 5.0 3.0 3.0 3....  
6  1.0 9.0 3.0 3.0 3.0 3.0 3.0 5.0 3.0 3.0 3.0 3....  
7  2.0 2.0 2.0 3.0 3.0 3.0 3.0 4.0 5.0 3.0 3.0 3....  
8  3.0 3.0 3.0 3.0 3.0 3.0 3.0 3.0 5.0 4.0 3.0 3....  
0     happiness
1       disgust
2      

In [153]:
import re
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# X_test['ApexFrame'] = X_test['ApexFrame'].str.strip("/")
# X_train['ApexFrame'] = X_train['ApexFrame'].str.strip("/")
# X_train['ApexFrame'] = np.where(X_train['ApexFrame'] == '', np.nan, X_train['ApexFrame'])

# Convert 'ApexFrame' to float, replacing NaN values with 0 (or any other desired value)
X_train['Computed LBP'] = pd.to_numeric(X_train['Computed LBP'], errors='coerce').fillna(0).astype(float)
X_test['Computed LBP'] = pd.to_numeric(X_test['Computed LBP'], errors='coerce').fillna(0).astype(float)

In [154]:
from pandas import unique


# print(X_test['ApexFrame'].unique())
# print(X_train['ApexFrame'].unique())


In [155]:
X_train.shape, X_test.shape

((7, 4), (2, 4))

In [160]:
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score
import joblib


# Classifier initialization
svm_classifier = SVC(kernel='linear', C=1.0)

# Training the Classifier
svm_classifier.fit(X_train, y_train)
joblib.dump(svm_classifier, '/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/svm_model_2.joblib')


['/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/svm_model_2.joblib']

In [161]:
trained_SVM_Model = joblib.load('/Users/jj/Documents/COLLEGE_DOCS/CASME2/SVM_Classifier_Model/svm_model_2.joblib')

# Test prediction using Test sample 
predictions = trained_SVM_Model.predict(X_test)

print(predictions)

accuracy = accuracy_score(y_test, predictions)
print (f"Accuracy: {accuracy * 100: .2f}%")

['repression' 'disgust']
Accuracy:  50.00%
